In [17]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import copy
import json
import geopandas as gpd
from arcgis.gis import GIS

In [2]:
from arcgis.geoenrichment import enrich, Country

In [3]:
# init GIS connection
with open("agol_account_info.txt", "r") as f:
    url, username, password = f.read().splitlines()

In [4]:
gis = GIS(url, username=username, password=password)

In [8]:
cal_bg = gpd.read_file('tl_2023_06_bg/tl_2023_06_bg.shp')

In [50]:
cal_bg

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,GEOIDFQ,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,025,940000,3,060259400003,1500000US060259400003,Block Group 3,G5030,S,12261656,685451,+32.7424027,-114.6675664,"POLYGON ((-114.72643 32.71857, -114.72639 32.7..."
1,06,025,010800,2,060250108002,1500000US060250108002,Block Group 2,G5030,S,59355860,0,+32.7658709,-115.3923999,"POLYGON ((-115.43082 32.73090, -115.43082 32.7..."
2,06,025,010800,1,060250108001,1500000US060250108001,Block Group 1,G5030,S,230625025,0,+32.8363507,-115.3514036,"POLYGON ((-115.45784 32.90895, -115.45773 32.9..."
3,06,025,011100,3,060250111003,1500000US060250111003,Block Group 3,G5030,S,1159586,0,+32.7932927,-115.6905731,"POLYGON ((-115.70134 32.79085, -115.70091 32.7..."
4,06,073,010013,2,060730100132,1500000US060730100132,Block Group 2,G5030,S,269440,0,+32.5536152,-117.0429162,"POLYGON ((-117.04560 32.55140, -117.04556 32.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25602,06,111,009300,2,061110093002,1500000US061110093002,Block Group 2,G5030,S,138980531,248409,+34.2976989,-119.0145578,"POLYGON ((-119.16134 34.26400, -119.15819 34.2..."
25603,06,111,009600,2,061110096002,1500000US061110096002,Block Group 2,G5030,S,24818163,35505,+34.3087281,-119.2278872,"POLYGON ((-119.26418 34.27879, -119.26313 34.2..."
25604,06,067,000501,1,060670005011,1500000US060670005011,Block Group 1,G5030,S,197592,0,+38.5859636,-121.4849213,"POLYGON ((-121.48842 38.58507, -121.48818 38.5..."
25605,06,067,000501,2,060670005012,1500000US060670005012,Block Group 2,G5030,S,197826,0,+38.5826810,-121.4863456,"POLYGON ((-121.48985 38.58179, -121.48961 38.5..."


In [51]:
imp_sd_bg = cal_bg[(cal_bg["COUNTYFP"] == "073") + (cal_bg["COUNTYFP"] == "025")][["STATEFP", "COUNTYFP", "TRACTCE", "BLKGRPCE", "geometry"]]

In [65]:
imp_sd_bg_sedf = pd.DataFrame.spatial.from_geodataframe(imp_sd_bg)

In [ ]:
url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/1'

In [ ]:
from arcgis.features import FeatureLayer
fl = FeatureLayer(url, gis)

In [ ]:
bg_sedf = pd.DataFrame.spatial.from_layer(fl)

In [32]:
usa = Country('usa')

In [15]:
sd_bgs_sedf = pd.DataFrame.spatial.from_layer(sd_bgs_enriched.layers[0])

In [37]:
# select current year key enrichment variables for analysis
ev_df = usa.enrich_variables

In [38]:
collection = ['Age', 'crime', 'educationalattainment', 'food', 'foodstampsSNAP', 'householdincome', 'industry', 'Wealth', 'occupation', 'OwnerRenter', 'RaceAndEthnicity', 'SpendingTotal', 'yearbuilt', 'yearmovedin']

In [39]:
kv_df  = ev_df[ev_df.apply(lambda row: row['data_collection'] in collection, axis =1)]

In [40]:
kv_df2328 = kv_df[kv_df.apply(lambda row: row['vintage'] in ['2023', '2028'], axis =1)]
kv_df_A = kv_df2328[kv_df2328.apply(lambda row: row["name"][-2:] == '_A', axis =1)]
kv_df_none = kv_df2328[kv_df2328.apply(lambda row: row["name"][-2] != '_', axis =1)]

In [41]:
kv_df2328A = pd.concat([kv_df_A, kv_df_none])

In [43]:
kv_df2328A

,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
6772,X2001_A,2023 Avg: Alcoholic Bevs,food,food.X2001_A,food_X2001_A,2023 Alcoholic Beverages: Average,2023,currency
6775,X2002_A,2023 Avg: Alcoholic Bevs at Home,food,food.X2002_A,food_X2002_A,2023 Alcoholic Beverages at Home: Average,2023,currency
6778,X2003_A,2023 Avg: Beer/Ale at Home,food,food.X2003_A,food_X2003_A,2023 Beer & Ale At Home: Average,2023,currency
6781,X2004_A,2023 Avg: Whiskey at Home,food,food.X2004_A,food_X2004_A,2023 Whiskey At Home: Average,2023,currency
6784,X2005_A,2023 Avg: Wine at Home,food,food.X2005_A,food_X2005_A,2023 Wine At Home: Average,2023,currency
...,...,...,...,...,...,...,...,...
20694,MEDVAL_FY,2028 Median Home Value,Wealth,Wealth.MEDVAL_FY,Wealth_MEDVAL_FY,2028 Median Home Value (Esri),2028,currency
20695,AVGVAL_FY,2028 Average Home Value,Wealth,Wealth.AVGVAL_FY,Wealth_AVGVAL_FY,2028 Average Home Value (Esri),2028,currency
20696,VALBASE_FY,2028 Home Value Base,Wealth,Wealth.VALBASE_FY,Wealth_VALBASE_FY,2028 Owner Occupied Housing Units by Value Bas...,2028,count
20697,WLTHINDXCY,2023 Wealth Index,Wealth,Wealth.WLTHINDXCY,Wealth_WLTHINDXCY,2023 Wealth Index (Esri),2023,count


In [71]:
kv_df2328A.to_csv("enrich_variables.csv")

In [66]:
imp_sd_bg_sedf

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,SHAPE
0,06,025,940000,3,"{""rings"": [[[-114.72643, 32.718573], [-114.726..."
1,06,025,010800,2,"{""rings"": [[[-115.430817, 32.730903], [-115.43..."
2,06,025,010800,1,"{""rings"": [[[-115.457835, 32.908953], [-115.45..."
3,06,025,011100,3,"{""rings"": [[[-115.701342, 32.790854], [-115.70..."
4,06,073,010013,2,"{""rings"": [[[-117.045596, 32.5514], [-117.0456..."
...,...,...,...,...,...
22531,06,073,017405,1,"{""rings"": [[[-117.281565, 33.036222], [-117.28..."
22532,06,073,007701,4,"{""rings"": [[[-117.240338, 32.791955], [-117.24..."
22533,06,073,021102,2,"{""rings"": [[[-116.279703, 32.633477], [-116.27..."
22534,06,073,013423,1,"{""rings"": [[[-116.985207, 32.670734], [-116.98..."


In [67]:
enrich_out = usa.enrich(imp_sd_bg_sedf, kv_df2328A)

C:\Users\Johnny Lei\Anaconda3\envs\nourish\lib\site-packages\arcgis\geoenrichment\_business_analyst\_main.py:1306: UserWarning: Some of the enrich_variables provided are not available for enrichment (missing count: 31).
  warn(


In [68]:
enrich_out

,statefp,countyfp,tractce,blkgrpce,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,x2001_a,...,val400k_fy,val500k_fy,val750k_fy,val1m_fy,medval_fy,avgval_fy,valbase_fy,wlthindxcy,sei_cy,SHAPE
0,06,025,940000,3,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,148.41,...,0.0,12.0,0.0,0.0,58333.0,142568.0,185.0,16.0,28.7,"{""rings"": [[[-114.72651900000238, 32.718274000..."
1,06,025,010800,2,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,668.78,...,41.0,15.0,3.0,0.0,359375.0,378230.0,209.0,99.0,58.2,"{""rings"": [[[-115.35666800000234, 32.722770000..."
2,06,025,010800,1,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,662.82,...,5.0,46.0,1.0,0.0,395455.0,428066.0,106.0,150.0,53.9,"{""rings"": [[[-115.32922200000232, 32.722710000..."
3,06,025,011100,3,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,257.18,...,0.0,94.0,3.0,0.0,185000.0,288768.0,276.0,28.0,33.6,"{""rings"": [[[-115.69995600000235, 32.788669000..."
4,06,073,010013,2,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,328.89,...,4.0,3.0,1.0,1.0,750000.0,1082292.0,24.0,33.0,37.7,"{""rings"": [[[-117.03984700000227, 32.549000000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,06,073,017405,1,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,755.09,...,2.0,14.0,161.0,112.0,950311.0,1006548.0,294.0,84.0,46.4,"{""rings"": [[[-117.27499200000224, 33.026307000..."
2168,06,073,007701,4,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,693.08,...,0.0,1.0,4.0,53.0,1231132.0,1222458.0,59.0,50.0,71.7,"{""rings"": [[[-117.2393210000022, 32.7883500000..."
2169,06,073,021102,2,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,335.29,...,168.0,3.0,3.0,6.0,427976.0,489516.0,310.0,37.0,51.8,"{""rings"": [[[-116.2709420000023, 32.6053680000..."
2170,06,073,013423,1,USA,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,983.91,...,11.0,113.0,325.0,34.0,841538.0,843145.0,496.0,173.0,68.4,"{""rings"": [[[-116.97140400000227, 32.663806000..."


In [69]:
lyr = enrich_out.spatial.to_featurelayer('bgs_sd_imp')
lyr

<Item title:"bgs_sd_imp" type:Feature Layer Collection owner:jil1119_UCSDOnline>

In [70]:
enrich_out.spatial.to_featureclass(location=r"bgs_sd_imp\bgs_sd_imp.shp")

'C:\\Users\\Johnny Lei\\Nourish\\bgs_sd_imp\\bgs_sd_imp.shp'